# Dataviz: Bokeh - PlotlyAPI

## Bokeh:  part 1

### I) dependencies

#### imports

In [5]:
import bokeh
from bokeh.io import output_file, show
from bokeh.layouts import gridplot
from bokeh.palettes import Viridis3
from bokeh.plotting import figure, Figure, output_file, show
from bokeh.layouts import widgetbox,column,Column
from bokeh.models.widgets import Button, RadioButtonGroup, Select, Slider
from bokeh.models import Circle,Line
from bokeh.models import HoverTool

from bokeh.models.formatters import DatetimeTickFormatter


from bokeh.sampledata.stocks import AAPL
from bokeh.layouts import layout
from bokeh.models import  Slider, ColumnDataSource, WidgetBox
from bokeh.models.callbacks import CustomJS
from math import pi

import pandas as pd
import numpy as np
import datetime

### II) Simple Graphs

#### scatter plots

In [ ]:
from bokeh.plotting import figure, output_file, show

# data series
x = [0.1, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
y0 = [i**2 for i in x]
y1 = [10**i for i in x]
y2 = [10**(i**2) for i in x]

# output to static HTML file
output_file("log_lines.html")

# create a new plot
p = figure(
   tools="pan,box_zoom,reset,save",
   y_axis_type="log", y_range=[0.001, 10**11], title="log axis example",
   x_axis_label='sections', y_axis_label='particles'
)

# add some renderers
p.line(x, x, legend="y=x")
p.circle(x, x, legend="y=x", fill_color="white", size=8)
p.line(x, y0, legend="y=x^2", line_width=3)
p.line(x, y1, legend="y=10^x", line_color="red")
p.circle(x, y1, legend="y=10^x", fill_color="red", line_color="red", size=6)
p.line(x, y2, legend="y=10^x^2", line_color="orange", line_dash="4 4")

# show the results
show(p)

#### Normal law plot

In [ ]:
mu, sigma = 0, 0.5

measured = np.random.normal(mu, sigma, 1000)
hist, edges = np.histogram(measured, density=True, bins=50)

In [ ]:
x = np.linspace(-2, 2, 1000)
pdf = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))

#Column Data source from which data will be fed
source = ColumnDataSource(data=dict(
left=edges[:-1],
right=edges[1:],
top=hist,

))

#tooltip
hover = HoverTool(tooltips=[
("index", "$index"),
("value","@top"),
("desc", "@left"),
])


#figure area with particular tools
p = figure(tools=['reset','box_zoom','save','pan',hover])

#histogram plot
p.quad(bottom=0,top='top',left='left',right='right',source=source,
       fill_color="#036564", line_color="#033649",legend="hist")
#line plot
p.line(x,pdf, line_color="#D95B43", line_width=4, line_alpha=0.4,legend="PDF")

#legend
p.legend.location = "top_right"

#miscellaneous
p.background_fill_color = "#E8DDCB"
p.background_fill_alpha = 0.4

show(p)
output_file('palette.html')

### III) Example 1:

#### dataset download

In [ ]:
try:
    bokeh.sampledata.download()
except:
    print("not good connection, timeout")

#### 2 top Graphs : close  and data (+ bands at 0.6.std) 

In [ ]:
df = pd.DataFrame(AAPL)

In [ ]:
df.head()

 ##### Quick and dirty way (not to be done)

In [ ]:
#Dataset
df = pd.DataFrame(AAPL)
df['date2'] = pd.to_datetime(df['date'])

#Output specification
output_file("dirty_way.html")


In [ ]:
# create a new plot with a datetime axis type
p = figure(plot_width=800, plot_height=250, x_axis_type="datetime")
p.title.text = "Simple graph close = f(t)"
p.line(df['date2'], df['close'], color='navy', alpha=0.5)

show(p)

#### The right way : Let's enclose the logic in 2 functions

In [ ]:
def stock_close():
    
    #Data conversion
    df = pd.DataFrame(AAPL)
    df['date2'] = pd.to_datetime(df['date'])

    #Column Data Source creation
    source = ColumnDataSource(data=dict(
    x=df['date2'],
    y=df['close'],
    z_plus=df['close']+0.6*np.array(df['close']).std(),
    z_moins=df['close']-0.6*np.array(df['close']).std(),
    desc=[el.strftime("%Y-%m-%d %H:%M:%S") for el in df['date2']]
    ))

    
    #Tooltip part
    hover = HoverTool(tooltips=[
    ("index", "$index"),
    ("value","$"+"@y"),
    ("desc", "@desc"),
    ])

 

    # create a new plot with a datetime axis type
    p1 = figure(plot_width=250, plot_height=250, x_axis_type="datetime",tools=['reset','box_zoom','save','pan',hover])
    p1.title.text = 'closing values (stock market)'
    
    p1.cross('x','y',line_color='navy', alpha=0.5,fill_color='white', line_width=1,name="close_data", source=source)
  
    p1.line( 'x','y',line_color='navy', alpha=0.1, line_width=0.5,source=source)
    p1.line('x','z_plus', color='grey', alpha=0.1,name="close_data_sup", source=source)
    p1.line('x','z_moins', color='grey', alpha=0.1,name="close_data_inf", source=source)

    p1.border_fill_color = "whitesmoke"

    return([p1])

In [ ]:
def stock_open():
    
    #Data conversion
    df = pd.DataFrame(AAPL)
    df['date2'] = pd.to_datetime(df['date'])
    
    #Data source part
    source = ColumnDataSource(data=dict(
    x=df['date2'],
    y=df['open'],
    z_plus=df['open']+0.6*np.array(df['open']).std(),
    z_moins=df['open']-0.6*np.array(df['open']).std(),
    desc=[el.strftime("%Y-%m-%d %H:%M:%S") for el in df['date2']]
    ))

    #Tooltip part
    hover = HoverTool(tooltips=[
    ("index", "$index"),
    ("value","$"+"@y"),
    ("desc", "@desc"),
    ])

    
    # create a new plot with a datetime axis type
    p2 = figure(plot_width=250, plot_height=250, x_axis_type="datetime",tools=['reset','box_zoom','save','pan',hover])
    p2.title.text = 'opening values (stock market)'
    p2.line( 'x','y',line_color='green', alpha=0.1, line_width=0.5,source=source)
    p2.cross('x','y',line_color='green', alpha=0.5,fill_color='white', line_width=1,name="open_data", source=source)
    p2.line('x','z_plus', color='grey', alpha=0.1,name="open_data_sup", source=source)
    p2.line('x','z_moins', color='grey', alpha=0.1,name="open_data_inf", source=source)

    p2.border_fill_color = "whitesmoke"

    return([p2])

In [ ]:
# 1 bottom Graph: difference between open and close values
# it allows us to emphasize the 2 main volatile events

In [ ]:
def close_open_diff():
    #Data conversion
    df = pd.DataFrame(AAPL)
    df['date2'] = pd.to_datetime(df['date'])
    desc=[el.strftime("%Y-%m-%d") for el in df['date2']]
    
    #colors range creation based on condition
    l=[]
    for i in range(len(df['open'])):
        if df['open'][i]>df['close'][i]:
            l.append("green")
        else:
            l.append("red") 
            
    #alpha range creation based on calculation 
    m=np.array(df['open']- df['close']).max()*0.5
    alphas=(df['open']-df['close'])/m
    
    #Data source part
    source2 = ColumnDataSource(data=dict(
    x=df['date2'] ,
    y=df['open']-df['close'],
    col=l,
    alphas=alphas,
    desc=desc
    ))
    
    #Tooltip part
    hover = HoverTool(tooltips=[
    ("index", "$index"),
    ("value","$"+"@y"),
    ("desc", "@desc"),
    ])

    #Graph construction part (with colors, alpha, x , y fed by ColumDataSource)
    p3 = figure(plot_width=500, plot_height=500, x_axis_type="datetime",tools=['reset','box_zoom','save','pan',hover])
    p3.title.text = 'open-close'

    p3.inverted_triangle('x', 'y',source=source2,fill_color=l, fill_alpha=alphas)

    p3.border_fill_color = "whitesmoke"

    return([p3])

In [ ]:
#final part: Layout displaying the graphs 
l = layout([
    stock_open(),stock_close(),close_open_diff()
], sizing_mode='stretch_both')
output_file("clean_way.html")
show(l)

### Example 2 : Let's define a moving average (with an ajustable window)

In [10]:
def MA(y,window_length=3):
    weights_list= np.repeat(1.0,window_length)/window_length
    result=np.convolve(weights_list,y[::-1],'valid')
    result=result[::-1]
    return(result)

In [11]:
def bollinger(window_of_ma=3):
    
    upperband = np.random.randint(0, 10,100)
    lowerband = upperband - 10
    x_data = np.arange(1, 101)
    window_of_ma = window_of_ma if window_of_ma >=0 else 1
    ma_= MA((upperband+lowerband)*0.5,window_of_ma)
    
    #Data source part
    source2 = ColumnDataSource(data=dict(
    x_= x_data[window_of_ma-1:] ,
    y1_= ma_,
    y2_=upperband[window_of_ma-1:],
    y3_=lowerband[window_of_ma-1:]
    ))
    
    #Tooltip part
    hover = HoverTool(tooltips=[
    ("index", "$index"),
    ("value ($) ","@y1_"),
    ("value upper-end ($) ","@y2_"),
    ("value lower-end ($) ","@y3_")  
    ])
    
    
    # Bollinger shading glyph:
    band_x = np.append(x_data, x_data[::-1])
    band_y = np.append(lowerband, upperband[::-1])

    p = figure(x_axis_type='datetime',tools=['box_zoom','pan','reset','save',hover])
    p.patch(band_x, band_y, color='#7570B3', fill_alpha=0.2,line_width=0.5)
    
    p.line('x_','y1_',source=source2, line_width=0.5, alpha=0.5, color='firebrick')
    
    p.title.text = 'Bollinger Bands'
    p.title_location = 'left'
    p.title.align = 'left'
    
    # this could have been defined at the figure level
    p.plot_height = 600
    p.plot_width = 800
    
    # miscellaneous
    p.grid.grid_line_alpha = 0.1
    p.yaxis.major_label_text_color = "darkturquoise"
    
    p.background_fill_color = "beige"
    p.background_fill_alpha = 0.1
    return [p]



In [12]:

def linked_panning():
    
    # calculation part
    angles=[]
    colors=[]
    N = 100
    x = np.linspace(0, 4 * np.pi, N)
    y1 = np.sin(x)
    y2 = np.cos(x)
    y3 = np.sin(x) + np.cos(x)
    for el in y2:
        if el<0:
            angles.append(pi/3)
            colors.append('firebrick')
        else:
            angles.append(0)
            colors.append('green')
            
     
    #Data source part
    source2 = ColumnDataSource(data=dict(
    x_= x ,
    y1_= y1,
    y2_= y2,
    y3_= y3,
    angles_= angles,
    colors_= colors,
    sizes_= 8*abs(y2)/y2.max(),
    alphas_=abs(y1)/y1.max()
    ))
    

    #Tooltip part
    #they can't be shared=> 1 tooltip per graph!
    hover = HoverTool(tooltips=[
    ("index", "$index"),
    ("value from graph 1 ","(€) "+"@y1_")
    ])
    
    hover2 = HoverTool(tooltips=[
    ("index", "$index"),
    ("value from graph 2 ","(£) "+"@y2_"),
    ("value from graph 3 ","($) "+"@y3_")])
    
    hover3 = HoverTool(tooltips=[
    ("index", "$index"),
    ("value from graph 1 ","(€) "+"@y1_"),
    ("value from graph 3 ","($) "+"@y3_")])
    
    def make_tools(x):
        tools_=['box_select','box_zoom','pan','reset','save',x]
        return tools_
     
    #Graph creation      
    #Graph 1: scatterplot
    s1 = figure(tools=make_tools(hover),plot_width=400, plot_height=400)
    s1.circle('x_', 'y1_',source=source2, color="orange", size=10, alpha='alphas_')
    s1.title.text = 'graph n°1'
    s1.title_location = 'left'
    s1.title.align = 'left'
    s1.background_fill_color = "beige"
    s1.background_fill_alpha = 0.1
    s1.yaxis.major_label_text_color = "darkturquoise"
    
    
    #Graph 2: scatterplot ( markers size and colors change with values)
    s2 = figure(tools=make_tools(hover2),x_range=s1.x_range, y_range=s1.y_range)
    s2.triangle('x_', 'y2_',source=source2, color='colors_', size='sizes_', alpha=0.5, angle='angles_')
    s2.title.text = 'graph n°2'
    s2.title_location = 'left'
    s2.title.align = 'left'
    s2.background_fill_color = "beige"
    s2.background_fill_alpha = 0.1
    s2.yaxis.major_label_text_color = "darkturquoise"


    #Graph 3: scatterplot
    s3 = figure(tools=make_tools(hover3),x_range=s1.x_range, y_range=s1.y_range)
    s3.cross('x_', 'y3_',source=source2, color="green", size=10, alpha=0.5)
    s3.title.text = 'graph n°3'
    s3.title_location = 'left'
    s3.title.align = 'left'
    s3.yaxis.major_label_text_color = "darkturquoise"
    s3.background_fill_color = "beige"
    s3.background_fill_alpha = 0.1
    
    #return(layout)
    return [s1, s2, s3]

In [13]:
l = layout([
    bollinger(),linked_panning()
], sizing_mode='stretch_both')
show(l)

### Example 3 : Let's make a plot with a widget (alpha version)

In [14]:
# at this point, one must refresh to see the changes taking action
# to be improved (unstable)

In [15]:
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.plotting import Figure, output_file, show

output_file("callback.html")

x = [x*0.005 for x in range(0, 200)]
y = np.repeat(0,200)

source = ColumnDataSource(data=dict(x=x, y=y))

plot = Figure(plot_width=400, plot_height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)
plot.title.text = 'translation  slider widget'


def callback(source=source, window=None):
    data = source.data
    mu = cb_obj.value
    x, y = data['x'], data['y']
    for i in range(len(x)):
        y[i] = mu/10
    source.change.emit()

slider = Slider(start=0.1, end=4, value=1, step=.1, title="translation",
                callback=CustomJS.from_py_func(callback))

layout = column(slider, plot)

show(layout)

INFO:bokeh.core.state:Session output file 'callback.html' already exists, will be overwritten.


## Plotly API:  part 2

### imports

In [16]:
import plotly 
import plotly.plotly as py
import plotly.graph_objs as go

#I usually do not put my credentials but for you to review and check:
#It is also used in the sklearn tensorflow tutorial
#I'm using a community subscription (very limited)=> can be upgraded ($).
# I removed the api_key and user name

plotly.tools.set_credentials_file(username='', api_key='')

### plots

#### Example 1

In [ ]:
# 4 series: x_theo,sincx,x,y

x_theo = np.linspace(-4, 4, 100)
sincx = np.sinc(x_theo)
x = [-3.8, -3.03, -1.91, -1.46, -0.89, -0.24, -0.0, 0.41, 0.89, 1.01, 1.91, 2.28, 2.79, 3.56]
y = [-0.02, 0.04, -0.01, -0.27, 0.36, 0.75, 1.03, 0.65, 0.28, 0.02, -0.11, 0.16, 0.04, -0.15]

#plot 1
trace1 = go.Scatter(
    x=x_theo,
    y=sincx,
    name='sinc(x)',
    
)
#plot 2
trace2 = go.Scatter(
    x=x,
    y=y,
    text= 'Trucmuche',
    mode='markers',
    name='calculated',
    error_y=dict(
        type='percent',
        value=10,
        color='#85144B',
        thickness=1.5,
        width=3,
        opacity=1,
        visible=True
    ),
    marker=dict(
        color='#85144B',
        size=5,
        
    )
)
# necessary to declare data as a list of plots
data = [trace1, trace2]

#plot in the notebook
py.iplot(data, filename='error-bar-style')